模块导入

In [127]:
import os
import requests
import pandas as pd
import numpy as np
import json
import time
import random
from lxml import etree


Request URL: 示例

    https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=0
    https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=20
    

In [139]:
# 生成url地址列表

def format_url(num):
    urls=[]
    base_url='https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start={}'
    for i in range(0,20*num,20):
        #url='https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=%{}'.format(i)
        url= base_url.format(i)   #此行代码与上一行一样效果。 在此用format形式，%d不行，因为地址中有多高url，无法识别传给那个一。
       # print(url)
        urls.append(url)
    return urls

urls=format_url(10)    # 这里爬取10页，可自行更爱函数输入值
print(urls)

['https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=0', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=20', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=40', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=60', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=80', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=100', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=120', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=140', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=160', 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%

伪装请求头

In [129]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'}

解析单页

In [130]:
def parse_base_info(url,headers):
    html = requests.get(url,headers=headers)
    bs = json.loads(html.text)
    df = pd.DataFrame()
    for i in bs['data']:
        casts = i['casts']  #主演
        cover = i['cover']  #海报
        directors = i['directors'] #导演
        m_id = i['id']  # ID
        rate = i['rate'] #评分
        star = i['star'] #标记人数
        title = i['title'] #片名
        url = i['url']  #网址
        cache = pd.DataFrame({'主演':[casts],'海报':[cover],'导演':[directors],
                              'ID':[m_id],'评分':[rate],'标记人数':[star],'片名':[title],'网址':[url]})
        df = pd.concat([df,cache])
                # 这个语法学习一下
                # 合并DataFrame ，默认axis=0 ，横向连接，追加行数。
    return df
    #print(bs['data'])

#parse_base_info('https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=0',headers)

In [131]:
print('Start : %s' % time.ctime())

time.sleep( 5 )     #time.sleep() 函数推迟调用线程的运行，可通过参数（秒数），表示进程挂起的时间。

print("End : %s" % time.ctime())

Start : Wed Aug 28 13:55:40 2019
End : Wed Aug 28 13:55:45 2019


循环批量爬取电影

In [140]:
result = pd.DataFrame()

count=1
for url in urls:
    df = parse_base_info(url,headers = headers)
    result = pd.concat([result,df])
    time.sleep(random.random()+2)
       # random() 方法返回随机生成的一个实数，它在[0,1)范围内。
    print('I had crawed page of:%d' %count)
    count+=1
    

I had crawed page of:1
I had crawed page of:2
I had crawed page of:3
I had crawed page of:4
I had crawed page of:5
I had crawed page of:6
I had crawed page of:7
I had crawed page of:8
I had crawed page of:9
I had crawed page of:10


In [133]:
result.head()

,主演,海报,导演,ID,评分,标记人数,片名,网址
0,"[徐峥, 王传君, 周一围, 谭卓, 章宇]",https://img3.doubanio.com/view/photo/s_ratio_p...,[文牧野],26752088,9.0,45,我不是药神,https://movie.douban.com/subject/26752088/
0,"[让·雷诺, 娜塔莉·波特曼, 加里·奥德曼, 丹尼·爱罗, 彼得·阿佩尔]",https://img3.doubanio.com/view/photo/s_ratio_p...,[吕克·贝松],1295644,9.4,45,这个杀手不太冷,https://movie.douban.com/subject/1295644/
0,"[蒂姆·罗宾斯, 摩根·弗里曼, 鲍勃·冈顿, 威廉姆·赛德勒, 克兰西·布朗]",https://img3.doubanio.com/view/photo/s_ratio_p...,[弗兰克·德拉邦特],1292052,9.7,50,肖申克的救赎,https://movie.douban.com/subject/1292052/
0,"[屈楚萧, 吴京, 李光洁, 吴孟达, 赵今麦]",https://img3.doubanio.com/view/photo/s_ratio_p...,[郭帆],26266893,7.9,40,流浪地球,https://movie.douban.com/subject/26266893/
0,"[汤姆·汉克斯, 罗宾·怀特, 加里·西尼斯, 麦凯尔泰·威廉逊, 莎莉·菲尔德]",https://img3.doubanio.com/view/photo/s_ratio_p...,[罗伯特·泽米吉斯],1292720,9.5,50,阿甘正传,https://movie.douban.com/subject/1292720/


后面还有获取单页详情部分

In [134]:
def parse_movie_info(url,headers = headers ,ip = ''):
    if ip == '':
        html = requests.get(url,headers =headers )
    else:
        html = requests.get(url,headers = headers,proxies = ip)
    bs = etree.HTML(html.text)
    #片名
    title = bs.xpath('//div[@id = "wrapper"]/div/h1/span')[0].text
    #上映时间
    year = bs.xpath('//div[@id = "wrapper"]/div/h1/span')[1].text
    # 电影类型
    m_type = []
    for m in bs.xpath('//span[@property="v:genre"]'):
        m_type.append(m.text)
    
    a = bs.xpath('//div[@id= "info"]')[0].xpath('string()')
    #片长
    m_time = bs.xpath('//span[@property="v:runtime"]')[0].text  # 遗留问题如何去掉分钟
    # m_time =a[a.find('片长: ') + 4:a.find('分钟\n')]  #时长

    #地区
    area = a[a.find('制片国家/地区:')+9:a.find('\n        语言')]  #地区
    
    try:
        #评分人数
        people = bs.xpath('//span[@property="v:votes"]')[0].text
        # 评分分布
        rating={}  #等级
        rate_count =bs.xpath('//div[@class="ratings-on-weight"]/div')
        for rate in rate_count:
            rating[rate.xpath('span/@title')[0]] = rate.xpath('span[@class = "rating_per"]')[0].text
    except:
        people ='None'
        rating={}
    
    #简介
    try:
        brief = bs.xpath('//span[@property="v:summary"]')[0].text.strip('\n \u3000')
    except:
        brief ='None'
    # 热评  如何抓的明天继续研究
    try:
        hot_comment = bs.xpath('//div[@id = "hot-comments"]/div/div/p/span')[0].text
    except:
        hot_comment = 'None'
    cache=pd.DataFrame({'片名':[title],'上映时间':[year],'电影类型':[m_type],'片长':[m_time],
                       '地区':[area],'评分人数':[people],'评分分布':[rating],'简介':[brief],'热评':[hot_comment],'网址':[url]})
    return cache
    

#单个测试   
#ip=''
#url='https://movie.douban.com/subject/26752088/'
#parse_movie_info(url,headers = headers,ip = ip)

批量访问单个电影页面

In [141]:
moive_result = pd.DataFrame()
ip = ''
count2 = 1
cw = 1

for url,name in zip(result['网址'].values,result['片名'].values):
    try:
        cache = parse_movie_info(url,headers = headers,ip =ip)
        movie_result = pd.concat([movie_result,cache])
        #movie_result = pd.concat([movie_result,cache])
        time.sleep(random.random())
        print('成功爬取了第%d部电影——%s'%(count2,name))
        count2+=1
    except:
        print('滴滴滴滴滴滴,第%d次报错'%cw)
        print('IP is:{}'.format(ip))
        cw+=1
        time.sleep(2)
        continue

成功爬取了第1部电影——我不是药神
成功爬取了第2部电影——这个杀手不太冷
成功爬取了第3部电影——肖申克的救赎
成功爬取了第4部电影——流浪地球
成功爬取了第5部电影——阿甘正传
成功爬取了第6部电影——复仇者联盟3：无限战争
成功爬取了第7部电影——盗梦空间
成功爬取了第8部电影——西虹市首富
成功爬取了第9部电影——泰坦尼克号
成功爬取了第10部电影——千与千寻
成功爬取了第11部电影——霸王别姬
成功爬取了第12部电影——三傻大闹宝莱坞
成功爬取了第13部电影——让子弹飞
成功爬取了第14部电影——怦然心动
成功爬取了第15部电影——摔跤吧！爸爸
成功爬取了第16部电影——疯狂动物城
成功爬取了第17部电影——毒液：致命守护者
成功爬取了第18部电影——忠犬八公的故事
成功爬取了第19部电影——一出好戏
成功爬取了第20部电影——当幸福来敲门
成功爬取了第21部电影——海上钢琴师
成功爬取了第22部电影——大话西游之大圣娶亲
成功爬取了第23部电影——海王
成功爬取了第24部电影——你的名字。
成功爬取了第25部电影——楚门的世界
成功爬取了第26部电影——阿凡达
成功爬取了第27部电影——星际穿越
成功爬取了第28部电影——少年派的奇幻漂流
成功爬取了第29部电影——头号玩家
成功爬取了第30部电影——无双
成功爬取了第31部电影——放牛班的春天
成功爬取了第32部电影——飞屋环游记
成功爬取了第33部电影——机器人总动员
成功爬取了第34部电影——寻梦环游记
成功爬取了第35部电影——龙猫
成功爬取了第36部电影——那些年，我们一起追的女孩
成功爬取了第37部电影——红海行动
成功爬取了第38部电影——大话西游之月光宝盒
成功爬取了第39部电影——初恋这件小事
成功爬取了第40部电影——无名之辈
成功爬取了第41部电影——无间道
成功爬取了第42部电影——天使爱美丽
成功爬取了第43部电影——复仇者联盟
成功爬取了第44部电影——碟中谍6：全面瓦解
成功爬取了第45部电影——剪刀手爱德华
成功爬取了第46部电影——战狼2
成功爬取了第47部电影——绿皮书
成功爬取了第48部电影——飞驰人生
成功爬取了第49部电影——美丽人生
成功爬取了第50部电影——唐人街探案2
成功爬取了第51部电影——罗马假日
成功爬取了第52部电影——唐伯虎点秋香
成功爬取了第5

In [142]:
movie_result

,片名,上映时间,电影类型,片长,地区,评分人数,评分分布,简介,热评,网址
0,恋恋笔记本 The Notebook,(2004),"[剧情, 爱情]",123分钟,美国,390141,"{'力荐': '43.2%', '推荐': '40.9%', '还行': '14.3%', ...",一个浸润着鲜红与纯白的爱情故事，写在一本洒满昏黄的笔记本上，被一位老先生(詹姆斯·加纳 饰)...,飞屋环游记前传,https://movie.douban.com/subject/1309163/
0,我不是药神,(2018),"[剧情, 喜剧]",117分钟,中国大陆,1202736,"{'力荐': '57.6%', '推荐': '34.8%', '还行': '6.9%', '...",普通中年男子程勇（徐峥 饰）经营着一家保健品店，失意又失婚。不速之客吕受益（王传君 饰）的到...,“你敢保证你一辈子不得病？”纯粹、直接、有力！常常感叹：电影只能是电影。但每看到这样的佳作，...,https://movie.douban.com/subject/26752088/
0,这个杀手不太冷 Léon,(1994),"[剧情, 动作, 犯罪]",110分钟(剧场版),法国,1414750,"{'力荐': '74.2%', '推荐': '22.4%', '还行': '3.2%', '...",里昂（让·雷诺饰）是名孤独的职业杀手，受人雇佣。一天，邻居家小姑娘马蒂尔达（纳塔丽·波特曼饰...,我开始想要过得快乐，睡在床上，有牵挂,https://movie.douban.com/subject/1295644/
0,肖申克的救赎 The Shawshank Redemption,(1994),"[剧情, 犯罪]",142分钟,美国,1571311,"{'力荐': '84.7%', '推荐': '13.7%', '还行': '1.5%', '...",20世纪40年代末，小有成就的青年银行家安迪（蒂姆·罗宾斯 Tim Robbins 饰）因涉...,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,https://movie.douban.com/subject/1292052/
0,流浪地球,(2019),"[科幻, 灾难]",125分钟,中国大陆,1298757,"{'力荐': '33.1%', '推荐': '37.7%', '还行': '22.0%', ...",近未来，科学家们发现太阳急速衰老膨胀，短时间内包括地球在内的整个太阳系都将被太阳所吞没。为了...,三星鼓励一下吧，四个字：太儿戏了。硬科幻和硬要科幻是两回事。,https://movie.douban.com/subject/26266893/
0,阿甘正传 Forrest Gump,(1994),"[剧情, 爱情]",142分钟,美国,1222755,"{'力荐': '76.1%', '推荐': '20.7%', '还行': '2.9%', '...",阿甘（汤姆·汉克斯 饰）于二战结束后不久出生在美国南方阿拉巴马州一个闭塞的小镇，他先天弱智，...,我生命里最温暖的一部电影,https://movie.douban.com/subject/1292720/
0,复仇者联盟3：无限战争 Avengers: Infinity War,(2018),"[动作, 科幻, 奇幻, 冒险]",149分钟,美国,607343,"{'力荐': '33.6%', '推荐': '43.7%', '还行': '19.5%', ...",最先与灭霸军团遭遇的雷神索尔一行遭遇惨烈打击，洛基遇害，空间宝石落入灭霸之手。未几，灭霸的先...,灭霸是一个有情怀的无产主义者，不惜牺牲此生最爱来整治这个混乱而拥挤不堪的宇宙，整治完了也不是...,https://movie.douban.com/subject/24773958/
0,盗梦空间 Inception,(2010),"[剧情, 科幻, 悬疑, 冒险]",148分钟,美国 / 英国,1208522,"{'力荐': '70.3%', '推荐': '25.3%', '还行': '4.0%', '...",道姆·柯布（莱昂纳多·迪卡普里奥 Leonardo DiCaprio 饰）与同事阿瑟（约瑟夫...,卡梅隆负责革新电影技术，诺兰负责革新电影结构...,https://movie.douban.com/subject/3541415/
0,西虹市首富,(2018),[喜剧],118分钟,中国大陆,692022,"{'力荐': '9.5%', '推荐': '29.1%', '还行': '44.4%', '...",西虹市丙级球队大翔队的守门员王多鱼（沈腾 饰）因比赛失利被教练开除，一筹莫展之际王多鱼突然收...,又名《当王思聪是怎样一种体验》，表面上是喜剧片，其实内在无比黑暗。\n有钱可以让最高傲强势的...,https://movie.douban.com/subject/27605698/
0,泰坦尼克号 Titanic,(1997),"[剧情, 爱情, 灾难]",194分钟,美国,1164639,"{'力荐': '73.0%', '推荐': '22.7%', '还行': '4.0%', '...",1912年4月10日，号称 “世界工业史上的奇迹”的豪华客轮泰坦尼克号开始了自己的处女航，从...,永远是我在电影院所看过最好的电影,https://movie.douban.com/subject/1292722/


文件存储

In [144]:
result.to_excel('C:\\Users\\Administrator\\Desktop\\python\\电影基本信息大全.xlsx')

In [145]:
movie_result.to_excel('C:\\Users\\Administrator\\Desktop\\python\\电影详细信息大全.xlsx')